In [1]:
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, GlobalMaxPooling1D, Dense
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = set(stopwords.words('english'))
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, Concatenate, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from bs4 import BeautifulSoup

2024-04-20 05:02:08.175584: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-20 05:02:08.175693: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-20 05:02:08.443839: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
import pandas as pd

# Try reading the CSV file with a different encoding
try:
    df = pd.read_csv("/kaggle/input/flipkart-product-review-dataset/flipkart_product.csv", encoding='latin1')
except UnicodeDecodeError as e:
    print("UnicodeDecodeError:", e)


In [3]:
df

,ProductName,Price,Rate,Review,Summary
0,Candes 12 L Room/Personal Air Cooler?ÿ?ÿ(White...,"??3,999",5,Super!,Great cooler.. excellent air flow and for this...
1,Candes 12 L Room/Personal Air Cooler?ÿ?ÿ(White...,"??3,999",5,Awesome,Best budget 2 fit cooler. Nice cooling
2,Candes 12 L Room/Personal Air Cooler?ÿ?ÿ(White...,"??3,999",3,Fair,The quality is good but the power of air is de...
3,Candes 12 L Room/Personal Air Cooler?ÿ?ÿ(White...,"??3,999",1,Useless product,Very bad product it's a only a fan
4,Candes 12 L Room/Personal Air Cooler?ÿ?ÿ(White...,"??3,999",3,Fair,Ok ok product
...,...,...,...,...,...
189869,"NIVEA Soft Light Moisturizer for Face, Hand & ...",â¹142,5,Terrific,Thanks
189870,"NIVEA Soft Light Moisturizer for Face, Hand & ...",â¹142,5,Terrific,Good
189871,"NIVEA Soft Light Moisturizer for Face, Hand & ...",â¹142,5,Just wow!,Super
189872,"NIVEA Soft Light Moisturizer for Face, Hand & ...",â¹142,4,Worth the money,Good


In [4]:
df.shape

(189874, 5)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 189874 entries, 0 to 189873
Data columns (total 5 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   ProductName  189874 non-null  object
 1   Price        189873 non-null  object
 2   Rate         189873 non-null  object
 3   Review       189870 non-null  object
 4   Summary      189860 non-null  object
dtypes: object(5)
memory usage: 7.2+ MB


In [6]:
df.describe()

,ProductName,Price,Rate,Review,Summary
count,189874,189873,189873,189870,189860
unique,812,561,9,1264,98343
top,MILTON Thermosteel Flip Lid 500 ml FlaskÂ Â (P...,"â¹1,299",5,Nan,Good
freq,9278,10005,108694,20862,14175


In [7]:
import pandas as pd

# Assuming 'df' is your DataFrame and 'column_name' is the name of the column you want to extract unique values from

# Get unique values from a specific column
unique_values = df['ProductName'].unique()

# Get the count of unique values
unique_count = len(unique_values)

# Print the count of unique values
print("Number of unique values:", unique_count)


Number of unique values: 812


In [8]:
# #   This is the categories which we are defining in our dataset.
# categories = {
#       "clothing": ["t-shirt","kurta","Jersey","marks & spencer","formals", "peter england", "pyjama sets", "girls'", "swin trunk", "swim", "trunk", "adidas", "nike", "puma", "shirt", "pants", "ring", "dress", "jeans", "shorts", "hoodie", "sweater", "underwear", "socks", "hat", "scarf", "gloves", "unisex", "men's", "suit", "tuxedo", "dress shirt", "tie", "cufflinks", "polo shirt", "chinos", "activewear", "joggers", "sweatpants", "workout shirt", "women's", "dress", "blouse", "skirt", "leggings", "activewear", "lingerie", "swimwear", "maternity", "maternity dress", "maternity pants", "nursing bra", "clothing", "shirt", "dress", "pants", "jeans", "shorts", "skirt", "jacket", "sweater", "coat", "socks", "underwear", "shoes", "sneakers", "sandals", "boots", "heels", "flats", "dress shoes", "accessories", "bag", "purse", "hat", "scarf", "belt", "jewelry", "watch", "sunglasses", "sandal", "sneaker" , "liberty" ,"slippers", "short", "sleeves","necklase", "necklace", "pyjamas","pyjama", "linen", "women", "men", "saree", "kurti","unisex", "uni-sex", "ring"],
#       "arts & crafts": ["paint","poster", "print", "brush","erasers", "staples", "stamp sheet", "canvas", "clay", "yarn", "needle", "bead", "glue", "paper", "craft kit", "watercolor", "acrylic", "easel", "palette","armature", "drawing", "pencil", "pen", "sketchpad", "marker", "drawing paper", "sculpting", "jewelry making", "beads", "findings", "pliers", "clasp"],
#       "automotive": ["car","sunroof", "helmet","crankshaft", "sensor", "tesla","exhaust", "truck", "mustang", "motorcycle","gasket", "engine", "tire", "battery", "yamaha", "filter", "light", "part", "brake", "wheel", "car part", "truck part", "motorcycle part", "engine part", "spark plug", "car cover", "floor mat", "seat cover", "phone mount", "sunshade", "car stereo", "speaker", "car wash", "wax", "polish", "degreaser", "tire shine", "interior cleaner", "windshield washer fluid", "steering", "hose" , "clutch" ,"radiator", "coolant", "suzuki", "horn", "maruti", "kia" , "chevy", "starter", "steelbird", "steel bird", "wisor", "car key", "key", "car accessories", "ring car", "mk7"," mk2", "ecosport", "strut", "assembly"],
#       "baby": ["diaper", "wipes", "bottle", "food", "crib", "stroller", "toy", "blanket", "clothes", "bath", "feeding", "diapering", "sleeping", "bath & grooming", "play", "nipple", "formula", "food pouch", "bib", "spoon", "highchair", "blender", "diaper rash cream", "changing pad", "wipes dispenser", "mattress", "bassinet", "sleep sack", "sound machine", "shampoo", "soap", "lotion", "washcloth", "baby brush", "rattle", "teether", "activity gym", "stuffed animal", "ball", "blocks", "playmat", "toodler", "child"],
#       "beauty & personal care": ["makeup","facial", "perfume", "costume", "scent", "skincare", "natural", "body", "scrub","haircare", "tooth", "toothbrush", "cream" , "face & body" , "face", "body","fragrance", "shaving", "bath", "soap", "lotion", "shampoo", "cosmetics", "foundation", "concealer", "powder", "blush", "eyeshadow", "mascara", "eyeliner", "lipstick", "lip gloss", "moisturizer", "cleanser", "toner", "serum", "face mask", "sunscreen", "eye cream", "perfume", "cologne", "body spray", "deodorant", "razor", "shaving cream", "aftershave", "beard oil", "trimmer", "electric shaver", "loreal paris", "paris"],
#       "books": ["book", "novel", "mathematics", "oxford","math","jerusalem", "canyon", "mystic", "concentration", "bullet", "physiology", "aliens","comic", "comics", "alien", "sword", "east", "west", "stories", "space", "nostalgic", "idiot","letters", "letter", "english", "hindi", "science", "history", "soul", "world", "edition","love", "strategy", "journal","myths", "myth", "legend", "jesus", "curse", "under", "courage", "die","bird", "river" "year", "souls", "step",  "mystery", "thriller", "romance", "fantasy", "science fiction", "historical fiction", "educational", "textbook", "workbook", "study guide", "manual", "reference book", "children's", "picture book", "board book", "chapter book", "young adult", "cookbook", "recipe book", "baking book", "healthy cooking", "how", "why", "who", "what", "the", " frozen", "poems", "poem", "love", "tips", "journey","wise", "words", "success", "dreams", "dream", "literature","politics"],
#       "electronics": ["television","seagate","ssd","hdd","purifier","nikon","cam","pcb","sony", "experia", "havells","flip cover"," honeywell", "cleaner", "d-link", "d link", "dlink", "design cover", "phone case", "lcd", "led" , "display", "screen","tv", "led tv", "oled tv", "smart tv", "projector", "screen", "computer", "laptop", "desktop", "monitor", "keyboard", "mouse", "webcam", "printer", "audio", "speaker", "headphone", "soundbar", "wireless speaker", "microphone", "phones & tablets", "smartphone", "tablet", "case", "charger", "screen protector", "appliances", "refrigerator", "stove", "microwave", "dishwasher", "coffee maker", "blender", "western digital", "500gb" , "gb" , "laptop" , "intel" , "ram" ,"rom" , "flash drive" , "ddr4" , "ddr3", "washing machine", "12v", "snapdragon", "qualcome" ,"camera", "back cover", "realme", "samsung", "redmi", "i phone", "mobile", "i pad", "tab", "printer", "mouse", "keyboard", "fpvdrone","DVD","dvd","USB","Kettle"],
#       "household": ["furniture", "cooking", "chopstick","chopper","grinder", "organiser","puja","bell", "straws" ,"mug","handle","stain-less steel", "curtain", "household", "bubble gum", "mandir", "tumbler", "drinking","serving", "slicer","clock", "wall", "decoration" , "cooler" ,"Geyser","Top Load", "inverter" , "tupperware" , "plastic container", "container", "dinner","Sewing Machine", "dinnerware", "statue", "pizza", "burger", "knife", "chocolate molding", "mattress", "bed frame", " under-bed","bed", "sofa", "chair", "table", "desk", "lamp", "bookshelf", "bedding", "sheets", "comforter", "pillowcase", "blanket", "mattress pad", "kitchenware", "pot", "pan", "knife", "cutting board", "spatula", "whisk", "mixer", "cleaning supplies", "vacuum cleaner", "mop", "broom", "dustpan", "sponge", "dish soap", "laundry detergent", "rug", "vase", "flower", "tissue box", "wall stticker", "wall", "floor", "hand towels", "towel", "cushion cover", "kitchen", "home", "curtain"],
#       "amazon originals": ["amazon", "amazon basics"], 
# #       "luggage": ["suitcase", "backpack", "duffel bag", "travel bag"],
#       "health": ["medicine", "vitamins", "cleaning supplies","capsules","nebulizer","sanitizer","skin", "first aid kit", "thermometer", "bandage"],
#       "pet supplies": ["dog food", "cat food", "leash", "collar", "bed", "toy", "fish food","cat litter", "birdseed", "cage", "aquarium"],
#       "software": ["operating system", "antivirus", "microsoft", "powerpoint", "autocad","productivity software", "design software", "game"],  # Consider subcategories
#       "sports & outdoors": ["sport", "exercise", "fitness","badmition", "yoga", "camping", "hiking", "fishing", "hunting", "bike", "tent", "sleeping bag", "backpack","Cricket Net"],  # Add keywords for specific sports equipment
#       "tools & home improvement": ["hammer","component", "lift kit","faucet","kohl","tape", "spacer", "insulator", "bracket", "wood", "pvc", "pcv valves", "valves", "screwdriver","tools","tool", "drill", "saw", "wrench", "paintbrush", "tape measure", "ladder", "toolbox", "screws", "nails", "paint", "hardware"],
#       "toys & games": ["toy", "game", "teddy bear", "doll","super", "mario", "collector", "sudoku","hot wheels", "hot wheel", "action figure", "puzzle", "board game", "building blocks", "stuffed animal", "car", "train", "video game","console", "game controller", "headset", "specific game titles", "ps5", "xbox", "ps4", "ps3"]# Consider including popular console names
#   }

In [9]:
# #   This is the categories which we are defining in our dataset.
# categories = {
#       "clothing": ["t-shirt","kurta","Jersey","marks & spencer","formals", "peter england", "pyjama sets", "girls'", "swin trunk", "swim", "trunk", "adidas", "nike", "puma", "shirt", "pants", "ring", "dress", "jeans", "shorts", "hoodie", "sweater", "underwear", "socks", "hat", "scarf", "gloves", "unisex", "men's", "suit", "tuxedo", "dress shirt", "tie", "cufflinks", "polo shirt", "chinos", "activewear", "joggers", "sweatpants", "workout shirt", "women's", "dress", "blouse", "skirt", "leggings", "activewear", "lingerie", "swimwear", "maternity", "maternity dress", "maternity pants", "nursing bra", "clothing", "shirt", "dress", "pants", "jeans", "shorts", "skirt", "jacket", "sweater", "coat", "socks", "underwear", "shoes", "sneakers", "sandals", "boots", "heels", "flats", "dress shoes", "accessories", "bag", "purse", "hat", "scarf", "belt", "jewelry", "watch", "sunglasses", "sandal", "sneaker" , "liberty" ,"slippers", "short", "sleeves","necklase", "necklace", "pyjamas","pyjama", "linen", "women", "men", "saree", "kurti","unisex", "uni-sex", "ring"],
#       "arts & crafts": ["paint","poster", "print", "brush","erasers", "staples", "stamp sheet", "canvas", "clay", "yarn", "needle", "bead", "glue", "paper", "craft kit", "watercolor", "acrylic", "easel", "palette","armature", "drawing", "pencil", "pen", "sketchpad", "marker", "drawing paper", "sculpting", "jewelry making", "beads", "findings", "pliers", "clasp"],
#       "automotive": ["car","sunroof", "helmet","crankshaft", "sensor", "tesla","exhaust", "truck", "mustang", "motorcycle","gasket", "engine", "tire", "battery", "yamaha", "filter", "light", "part", "brake", "wheel", "car part", "truck part", "motorcycle part", "engine part", "spark plug", "car cover", "floor mat", "seat cover", "phone mount", "sunshade", "car stereo", "speaker", "car wash", "wax", "polish", "degreaser", "tire shine", "interior cleaner", "windshield washer fluid", "steering", "hose" , "clutch" ,"radiator", "coolant", "suzuki", "horn", "maruti", "kia" , "chevy", "starter", "steelbird", "steel bird", "wisor", "car key", "key", "car accessories", "ring car", "mk7"," mk2", "ecosport", "strut", "assembly"],
#       "baby": ["diaper", "wipes", "bottle", "food", "crib", "stroller", "toy", "blanket", "clothes", "bath", "feeding", "diapering", "sleeping", "bath & grooming", "play", "nipple", "formula", "food pouch", "bib", "spoon", "highchair", "blender", "diaper rash cream", "changing pad", "wipes dispenser", "mattress", "bassinet", "sleep sack", "sound machine", "shampoo", "soap", "lotion", "washcloth", "baby brush", "rattle", "teether", "activity gym", "stuffed animal", "ball", "blocks", "playmat", "toodler", "child"],
#       "beauty & personal care": ["makeup","facial", "perfume", "costume", "scent", "skincare", "natural", "body", "scrub","haircare", "tooth", "toothbrush", "cream" , "face & body" , "face", "body","fragrance", "shaving", "bath", "soap", "lotion", "shampoo", "cosmetics", "foundation", "concealer", "powder", "blush", "eyeshadow", "mascara", "eyeliner", "lipstick", "lip gloss", "moisturizer", "cleanser", "toner", "serum", "face mask", "sunscreen", "eye cream", "perfume", "cologne", "body spray", "deodorant", "razor", "shaving cream", "aftershave", "beard oil", "trimmer", "electric shaver", "loreal paris", "paris"],
#       "books": ["book", "novel", "mathematics", "oxford","math","jerusalem", "canyon", "mystic", "concentration", "bullet", "physiology", "aliens","comic", "comics", "alien", "sword", "east", "west", "stories", "space", "nostalgic", "idiot","letters", "letter", "english", "hindi", "science", "history", "soul", "world", "edition","love", "strategy", "journal","myths", "myth", "legend", "jesus", "curse", "under", "courage", "die","bird", "river" "year", "souls", "step",  "mystery", "thriller", "romance", "fantasy", "science fiction", "historical fiction", "educational", "textbook", "workbook", "study guide", "manual", "reference book", "children's", "picture book", "board book", "chapter book", "young adult", "cookbook", "recipe book", "baking book", "healthy cooking", "how", "why", "who", "what", "the", " frozen", "poems", "poem", "love", "tips", "journey","wise", "words", "success", "dreams", "dream", "literature","politics"],
#       "electronics": ["television","seagate","ssd","HDMI","Streaming Device","Wireless Router","hdd","purifier","nikon","cam","pcb","sony", "experia", "havells","flip cover"," honeywell", "cleaner", "d-link", "d link", "dlink", "design cover", "phone case", "lcd", "led" , "display", "screen","tv", "led tv", "oled tv", "smart tv", "projector", "screen", "computer", "laptop", "desktop", "monitor", "keyboard", "mouse", "webcam", "printer", "audio", "speaker", "headphone", "soundbar", "wireless speaker", "microphone", "phones & tablets", "smartphone", "tablet", "case", "charger", "screen protector", "appliances", "refrigerator", "stove", "microwave", "dishwasher", "coffee maker", "blender", "western digital", "500gb" , "gb" , "laptop" , "intel" , "ram" ,"rom" , "flash drive" , "ddr4" , "ddr3", "washing machine", "12v", "snapdragon", "qualcome" ,"camera", "back cover", "realme", "samsung", "redmi", "i phone", "mobile", "i pad", "tab", "printer", "mouse", "keyboard", "fpvdrone","DVD","dvd","USB","Kettle"],
#       "household": ["furniture", "cooking", "chopstick","chopper","grinder", "organiser","puja","bell", "straws" ,"mug","handle","stain-less steel", "curtain", "household", "bubble gum", "mandir", "tumbler", "drinking","serving", "slicer","clock", "wall", "decoration" , "cooler" , "inverter" , "tupperware" , "plastic container", "container", "dinner", "dinnerware", "statue", "pizza", "burger", "knife", "chocolate molding", "mattress", "bed frame", " under-bed","bed", "sofa", "chair", "table", "desk", "lamp", "bookshelf", "bedding", "sheets", "comforter", "pillowcase", "blanket", "mattress pad", "kitchenware", "pot", "pan", "knife", "cutting board", "spatula", "whisk", "mixer", "cleaning supplies", "vacuum cleaner", "mop", "broom", "dustpan", "sponge", "dish soap", "laundry detergent", "rug", "vase", "flower", "tissue box", "wall stticker", "wall", "floor", "hand towels", "towel", "cushion cover", "kitchen", "home", "curtain","Cleaning Cloth"],
#       "amazon originals": ["amazon", "amazon basics"], 
# #       "luggage": ["suitcase", "backpack", "duffel bag", "travel bag"],
#       "health": ["medicine", "vitamins", "cleaning supplies","capsules","nebulizer","sanitizer","skin", "first aid kit", "thermometer", "bandage"],
#       "pet supplies": ["dog food", "cat food", "leash", "collar", "bed", "toy", "fish food","cat litter", "birdseed", "cage", "aquarium"],
#       "software": ["operating system", "antivirus", "microsoft", "powerpoint", "autocad","productivity software", "design software", "game"],  # Consider subcategories
#       "sports & outdoors": ["sport", "exercise", "fitness","badmition", "yoga", "camping", "hiking", "fishing", "hunting", "bike", "tent", "sleeping bag", "backpack"],  # Add keywords for specific sports equipment
#       "tools & home improvement": ["hammer","component", "lift kit","faucet","kohl","tape", "spacer", "insulator", "bracket", "wood", "pvc", "pcv valves", "valves", "screwdriver","tools","tool", "drill", "saw", "wrench", "paintbrush", "tape measure", "ladder", "toolbox", "screws", "nails", "paint", "hardware"],
#       "toys & games": ["toy", "game", "teddy bear", "doll","super", "mario", "collector", "sudoku","hot wheels", "hot wheel", "action figure", "puzzle", "board game", "building blocks", "stuffed animal", "car", "train", "video game","console", "game controller", "headset", "specific game titles", "ps5", "xbox", "ps4", "ps3"]# Consider including popular console names
#   }

In [10]:
#   This is the categories which we are defining in our dataset.
categories = {
      "clothing": ["t-shirt","kurta","Jersey","marks & spencer","formals", "peter england", "pyjama sets", "girls'", "swin trunk", "swim", "trunk", "adidas", "nike", "puma", "shirt", "pants", "ring", "dress", "jeans", "shorts", "hoodie", "sweater", "underwear", "socks", "hat", "scarf", "gloves", "unisex", "men's", "suit", "tuxedo", "dress shirt", "tie", "cufflinks", "polo shirt", "chinos", "activewear", "joggers", "sweatpants", "workout shirt", "women's", "dress", "blouse", "skirt", "leggings", "activewear", "lingerie", "swimwear", "maternity", "maternity dress", "maternity pants", "nursing bra", "clothing", "shirt", "dress", "pants", "jeans", "shorts", "skirt", "jacket", "sweater", "coat", "socks", "underwear", "shoes", "sneakers", "sandals", "boots", "heels", "flats", "dress shoes", "accessories", "bag", "purse", "hat", "scarf", "belt", "jewelry", "watch", "sunglasses", "sandal", "sneaker" , "liberty" ,"slippers", "short", "sleeves","necklase", "necklace", "pyjamas","pyjama", "linen", "women", "men", "saree", "kurti","unisex", "uni-sex", "ring"],
      "arts & crafts": ["paint","poster", "print", "brush","erasers", "staples", "stamp sheet", "canvas", "clay", "yarn", "needle", "bead", "glue", "paper", "craft kit", "watercolor", "acrylic", "easel", "palette","armature", "drawing", "pencil", "pen", "sketchpad", "marker", "drawing paper", "sculpting", "jewelry making", "beads", "findings", "pliers", "clasp"],
      "automotive": ["car","sunroof", "helmet","crankshaft", "sensor", "tesla","exhaust", "truck", "mustang", "motorcycle","gasket", "engine", "tire", "battery", "yamaha", "filter", "light", "part", "brake", "wheel", "car part", "truck part", "motorcycle part", "engine part", "spark plug", "car cover", "floor mat", "seat cover", "phone mount", "sunshade", "car stereo", "speaker", "car wash", "wax", "polish", "degreaser", "tire shine", "interior cleaner", "windshield washer fluid", "steering", "hose" , "clutch" ,"radiator", "coolant", "suzuki", "horn", "maruti", "kia" , "chevy", "starter", "steelbird", "steel bird", "wisor", "car key", "key", "car accessories", "ring car", "mk7"," mk2", "ecosport", "strut", "assembly"],
      "baby": ["diaper", "wipes", "bottle", "food", "crib", "stroller", "toy", "baby blanket", "clothes", "bath", "feeding", "diapering", "sleeping", "bath & grooming", "play", "nipple", "formula", "food pouch", "bib", "spoon", "highchair", "blender", "diaper rash cream", "changing pad", "wipes dispenser", "baby", "bassinet", "sleep sack", "sound machine", "shampoo", "soap", "lotion", "washcloth", "baby brush", "rattle", "teether", "activity gym", "stuffed animal", "ball", "blocks", "playmat", "toodler", "child"],
      "beauty & personal care": ["makeup","facial", "perfume","cosmetic", "costume","jewelery","scent", "skincare", "natural", "body", "scrub","haircare", "tooth", "toothbrush", "cream" , "face & body" , "face", "body","fragrance", "shaving", "bath", "soap", "lotion", "shampoo", "cosmetics", "foundation", "concealer", "powder", "blush", "eyeshadow", "mascara", "eyeliner", "lipstick", "lip gloss", "moisturizer", "cleanser", "toner", "serum", "face mask", "sunscreen", "eye cream", "perfume", "cologne", "body spray", "deodorant", "razor", "shaving cream", "aftershave", "beard oil", "trimmer", "electric shaver", "loreal paris", "paris"],
      "books": ["book", "novel", "mathematics", "oxford","math","jerusalem", "canyon", "mystic", "concentration", "bullet", "physiology", "aliens","comic", "comics", "alien", "sword", "east", "west", "stories", "space", "nostalgic", "idiot","letters", "letter", "english", "hindi", "science", "history", "soul", "world", "edition","love", "strategy", "journal","myths", "myth", "legend", "jesus", "curse", "under", "courage", "die","bird", "river" "year", "souls", "step",  "mystery", "thriller", "romance", "fantasy", "science fiction", "historical fiction", "educational", "textbook", "workbook", "study guide", "manual", "reference book", "children's", "picture book", "board book", "chapter book", "young adult", "cookbook", "recipe book", "baking book", "healthy cooking", "how", "why", "who", "what", "the", " frozen", "poems", "poem", "love", "tips", "journey","wise", "words", "success", "dreams", "dream", "literature","politics"],
      "electronics": ["television","seagate","ssd","hdmi","adapter","power adapter","dth","usb","streaming device","wifi","power bank","power adapter","catridge","Streaming Device","setup box","router","Wireless Router","hdd","purifier","nikon","cam","pcb","sony", "experia", "havells","flip cover"," honeywell", "cleaner", "d-link", "d link", "dlink", "design cover", "phone case", "lcd", "led" , "display", "screen","tv", "led tv", "oled tv", "smart tv", "projector", "screen", "computer", "laptop", "desktop", "monitor", "keyboard", "mouse", "webcam", "printer", "audio", "speaker", "headphone", "soundbar", "wireless speaker", "microphone", "phones & tablets", "smartphone", "tablet", "case", "charger", "screen protector", "appliances", "refrigerator", "stove", "microwave", "dishwasher", "coffee maker", "blender", "western digital", "500gb" , "gb" , "laptop" , "intel" , "ram" ,"rom" , "flash drive" , "ddr4" , "ddr3", "washing machine", "12v", "snapdragon", "qualcome" ,"camera", "back cover", "realme", "samsung", "redmi", "i phone", "mobile", "i pad", "tab", "printer", "mouse", "keyboard", "fpvdrone","dvd","dvd","usb","accessory"],
      "household": ["furniture", "cooking", "chopstick","chopper","otg","oven","grinder", "organiser","shelf","basket","cooker","puja","bell","sewing", "straws" ,"mug","handle","stain-less steel","ceiling fan","incense","door mat", "curtain", "household", "bubble gum", "mandir", "tumbler", "drinking","serving",'wardrobe', "slicer","clock", "wall", "decoration" ,"mattress", "cooler","trap","garden" , "inverter" , "tupperware" , "plastic container", "blanket","container", "dinner", "dinnerware", "statue", "pizza", "burger", "knife", "chocolate molding", "mattress","cushion", "bed frame", " under-bed","bed", "sofa", "chair", "table", "desk", "lamp", "bookshelf", "bedding", "sheets", "comforter", "pillowcase", "blanket", "mattress pad", "kitchenware", "pot", "pan", "knife", "cutting board", "spatula", "whisk", "mixer", "cleaning supplies", "vacuum cleaner", "mop", "broom", "dustpan", "sponge", "dish soap", "laundry detergent", "rug", "vase", "flower", "tissue box", "wall stticker", "wall", "floor", "hand towels", "towel", "cushion cover", "kitchen", "home", "curtain","microfiber","cleaning cloth","duster","electric kettle","kettle","heater rod","iron","juicer","geyser","automatic top load"],
      "amazon originals": ["amazon", "amazon basics"], 
#       "luggage": ["suitcase", "backpack", "duffel bag", "travel bag"],
      "health": ["medicine", "vitamins", "cleaning supplies","capsules","nebulizer","sanitizer","skin", "first aid kit", "thermometer", "bandage","ankle support","balm","nutrition drink","antiseptic"],
      "pet supplies": ["dog food", "cat food", "leash", "collar", "bed", "toy", "fish food","cat litter", "birdseed", "cage", "aquarium"],
      "software": ["operating system", "antivirus", "microsoft", "powerpoint", "autocad","productivity software", "design software", "game"],  # Consider subcategories
      "sports & outdoors": ["sport", "exercise", "fitness","badmition", "yoga", "camping", "hiking", "fishing", "hunting", "bike", "tent", "sleeping bag", "backpack","cricket","cycle"],  # Add keywords for specific sports equipment
      "tools & home improvement": ["hammer","component", "lift kit","faucet","kohl","tape", "spacer", "insulator", "bracket", "wood", "pvc", "pcv valves", "valves", "screwdriver","tools","tool", "drill", "saw", "wrench", "paintbrush", "tape measure", "ladder", "toolbox", "screws", "nails", "paint", "hardware",'seed',"tree","plant"],
      "toys & games": ["toy", "game", "teddy bear", "doll","super","rc", "mario", "collector", "sudoku","hot wheels", "hot wheel", "action figure", "puzzle", "board game", "building blocks", "stuffed animal", "car", "train", "video game","console", "game controller", "headset", "specific game titles", "ps5", "xbox", "ps4", "ps3"]# Consider including popular console names
  }

In [11]:
# This is the function which is traversing to all the titles and extracting the categories from the title using keywords

import pandas as pd

def categorize_product(data):

  # Add a new column named "Category" for categories 
  data["Category"] = "Uncategorized"

  # Loop through rows (products) in the DataFrame
  for index, row in data.iterrows():
    title = row["ProductName"]  
    if isinstance(title, str):  
      title = title.lower()
    else:
      print(f"Warning: Invalid title type (not string) at row {index}")  # Optional: Warn about invalid types
      title = ""  # Or assign an empty string for processing (replace with a more suitable default if needed)
    category = "Uncategorized"
    matches = 0

    for cat, keywords in categories.items():
      num_matches = sum(keyword in title for keyword in keywords)
      if num_matches > matches:
        matches = num_matches
        category = cat

    data.at[index, "Category"] = category

  return data

categorized_data = categorize_product(df.copy())
print(categorized_data)


                                              ProductName     Price Rate  \
0       Candes 12 L Room/Personal Air Cooler?ÿ?ÿ(White...  ??3,999    5   
1       Candes 12 L Room/Personal Air Cooler?ÿ?ÿ(White...  ??3,999    5   
2       Candes 12 L Room/Personal Air Cooler?ÿ?ÿ(White...  ??3,999    3   
3       Candes 12 L Room/Personal Air Cooler?ÿ?ÿ(White...  ??3,999    1   
4       Candes 12 L Room/Personal Air Cooler?ÿ?ÿ(White...  ??3,999    3   
...                                                   ...       ...  ...   
189869  NIVEA Soft Light Moisturizer for Face, Hand & ...    â¹142    5   
189870  NIVEA Soft Light Moisturizer for Face, Hand & ...    â¹142    5   
189871  NIVEA Soft Light Moisturizer for Face, Hand & ...    â¹142    5   
189872  NIVEA Soft Light Moisturizer for Face, Hand & ...    â¹142    4   
189873                                                         NaN  NaN   

                 Review                                            Summary  \
0        

In [12]:
# Assuming 'df' is your DataFrame with the added "Category" column
uncategorized_count = (categorized_data['Category'] == "Uncategorized").sum()
print(f"Number of Uncategorized items: {uncategorized_count}")

Number of Uncategorized items: 1


In [13]:
categorized_data.to_csv("categorized_products.csv", index=False)

In [14]:
# Loading new updated dataset
new_data = pd.read_csv("/kaggle/working/categorized_products.csv")

In [15]:
new_data

,ProductName,Price,Rate,Review,Summary,Category
0,Candes 12 L Room/Personal Air Cooler?ÿ?ÿ(White...,"??3,999",5,Super!,Great cooler.. excellent air flow and for this...,household
1,Candes 12 L Room/Personal Air Cooler?ÿ?ÿ(White...,"??3,999",5,Awesome,Best budget 2 fit cooler. Nice cooling,household
2,Candes 12 L Room/Personal Air Cooler?ÿ?ÿ(White...,"??3,999",3,Fair,The quality is good but the power of air is de...,household
3,Candes 12 L Room/Personal Air Cooler?ÿ?ÿ(White...,"??3,999",1,Useless product,Very bad product it's a only a fan,household
4,Candes 12 L Room/Personal Air Cooler?ÿ?ÿ(White...,"??3,999",3,Fair,Ok ok product,household
...,...,...,...,...,...,...
189869,"NIVEA Soft Light Moisturizer for Face, Hand & ...",â¹142,5,Terrific,Thanks,beauty & personal care
189870,"NIVEA Soft Light Moisturizer for Face, Hand & ...",â¹142,5,Terrific,Good,beauty & personal care
189871,"NIVEA Soft Light Moisturizer for Face, Hand & ...",â¹142,5,Just wow!,Super,beauty & personal care
189872,"NIVEA Soft Light Moisturizer for Face, Hand & ...",â¹142,4,Worth the money,Good,beauty & personal care


In [16]:
uncategorized_items = categorized_data[categorized_data['Category'] == "Uncategorized"]
print(uncategorized_items)

# Assuming 'uncategorized_items' is your DataFrame containing uncategorized items
uncategorized_items.to_csv('/kaggle/working/uncategorized_items.csv', index=False)


       ProductName Price Rate Review Summary       Category
189873              NaN  NaN    NaN     NaN  Uncategorized


In [17]:
import pandas as pd

# Assuming 'df' is your DataFrame and 'column_name' is the name of the column you want to convert to lowercase

# Convert the entire column to lowercase
new_data['Summary'] = df['Summary'].str.lower()

# Print the DataFrame to verify the changes
print(new_data)


                                              ProductName     Price Rate  \
0       Candes 12 L Room/Personal Air Cooler?ÿ?ÿ(White...  ??3,999    5   
1       Candes 12 L Room/Personal Air Cooler?ÿ?ÿ(White...  ??3,999    5   
2       Candes 12 L Room/Personal Air Cooler?ÿ?ÿ(White...  ??3,999    3   
3       Candes 12 L Room/Personal Air Cooler?ÿ?ÿ(White...  ??3,999    1   
4       Candes 12 L Room/Personal Air Cooler?ÿ?ÿ(White...  ??3,999    3   
...                                                   ...       ...  ...   
189869  NIVEA Soft Light Moisturizer for Face, Hand & ...    â¹142    5   
189870  NIVEA Soft Light Moisturizer for Face, Hand & ...    â¹142    5   
189871  NIVEA Soft Light Moisturizer for Face, Hand & ...    â¹142    5   
189872  NIVEA Soft Light Moisturizer for Face, Hand & ...    â¹142    4   
189873                                                         NaN  NaN   

                 Review                                            Summary  \
0        

In [18]:
import pandas as pd

# Assuming 'df' is your DataFrame and 'column_name' is the name of the column you want to process

# Remove non-word and non-whitespace characters from the entire column
new_data['Summary'] = df['Summary'].str.replace(r'[^\w\s]', '', regex=True)

# Print the DataFrame to verify the changes
print(new_data)


                                              ProductName     Price Rate  \
0       Candes 12 L Room/Personal Air Cooler?ÿ?ÿ(White...  ??3,999    5   
1       Candes 12 L Room/Personal Air Cooler?ÿ?ÿ(White...  ??3,999    5   
2       Candes 12 L Room/Personal Air Cooler?ÿ?ÿ(White...  ??3,999    3   
3       Candes 12 L Room/Personal Air Cooler?ÿ?ÿ(White...  ??3,999    1   
4       Candes 12 L Room/Personal Air Cooler?ÿ?ÿ(White...  ??3,999    3   
...                                                   ...       ...  ...   
189869  NIVEA Soft Light Moisturizer for Face, Hand & ...    â¹142    5   
189870  NIVEA Soft Light Moisturizer for Face, Hand & ...    â¹142    5   
189871  NIVEA Soft Light Moisturizer for Face, Hand & ...    â¹142    5   
189872  NIVEA Soft Light Moisturizer for Face, Hand & ...    â¹142    4   
189873                                                         NaN  NaN   

                 Review                                            Summary  \
0        

In [19]:
new_data

,ProductName,Price,Rate,Review,Summary,Category
0,Candes 12 L Room/Personal Air Cooler?ÿ?ÿ(White...,"??3,999",5,Super!,Great cooler excellent air flow and for this p...,household
1,Candes 12 L Room/Personal Air Cooler?ÿ?ÿ(White...,"??3,999",5,Awesome,Best budget 2 fit cooler Nice cooling,household
2,Candes 12 L Room/Personal Air Cooler?ÿ?ÿ(White...,"??3,999",3,Fair,The quality is good but the power of air is de...,household
3,Candes 12 L Room/Personal Air Cooler?ÿ?ÿ(White...,"??3,999",1,Useless product,Very bad product its a only a fan,household
4,Candes 12 L Room/Personal Air Cooler?ÿ?ÿ(White...,"??3,999",3,Fair,Ok ok product,household
...,...,...,...,...,...,...
189869,"NIVEA Soft Light Moisturizer for Face, Hand & ...",â¹142,5,Terrific,Thanks,beauty & personal care
189870,"NIVEA Soft Light Moisturizer for Face, Hand & ...",â¹142,5,Terrific,Good,beauty & personal care
189871,"NIVEA Soft Light Moisturizer for Face, Hand & ...",â¹142,5,Just wow!,Super,beauty & personal care
189872,"NIVEA Soft Light Moisturizer for Face, Hand & ...",â¹142,4,Worth the money,Good,beauty & personal care


In [20]:
import pandas as pd

# Assuming 'df' is your DataFrame and 'column_name' is the name of the column you want to process

# Remove digits from the entire column
new_data['Summary'] = df['Summary'].str.replace(r'\d+', '', regex=True)

# Print the DataFrame to verify the changes
print(new_data)


                                              ProductName     Price Rate  \
0       Candes 12 L Room/Personal Air Cooler?ÿ?ÿ(White...  ??3,999    5   
1       Candes 12 L Room/Personal Air Cooler?ÿ?ÿ(White...  ??3,999    5   
2       Candes 12 L Room/Personal Air Cooler?ÿ?ÿ(White...  ??3,999    3   
3       Candes 12 L Room/Personal Air Cooler?ÿ?ÿ(White...  ??3,999    1   
4       Candes 12 L Room/Personal Air Cooler?ÿ?ÿ(White...  ??3,999    3   
...                                                   ...       ...  ...   
189869  NIVEA Soft Light Moisturizer for Face, Hand & ...    â¹142    5   
189870  NIVEA Soft Light Moisturizer for Face, Hand & ...    â¹142    5   
189871  NIVEA Soft Light Moisturizer for Face, Hand & ...    â¹142    5   
189872  NIVEA Soft Light Moisturizer for Face, Hand & ...    â¹142    4   
189873                                                         NaN  NaN   

                 Review                                            Summary  \
0        

In [21]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize

# Assuming 'df' is your DataFrame and 'column_name' is the name of the column you want to tokenize

# Download NLTK resources if necessary (you only need to do this once)
nltk.download('punkt')

# Ensure all values in the column are strings
new_data['Summary'] = new_data['Summary'].astype(str)

# Tokenize the text in the column
new_data['Summary'] = new_data['Summary'].apply(lambda x: word_tokenize(x))

# Print the DataFrame to verify the changes
print(new_data)


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
                                              ProductName     Price Rate  \
0       Candes 12 L Room/Personal Air Cooler?ÿ?ÿ(White...  ??3,999    5   
1       Candes 12 L Room/Personal Air Cooler?ÿ?ÿ(White...  ??3,999    5   
2       Candes 12 L Room/Personal Air Cooler?ÿ?ÿ(White...  ??3,999    3   
3       Candes 12 L Room/Personal Air Cooler?ÿ?ÿ(White...  ??3,999    1   
4       Candes 12 L Room/Personal Air Cooler?ÿ?ÿ(White...  ??3,999    3   
...                                                   ...       ...  ...   
189869  NIVEA Soft Light Moisturizer for Face, Hand & ...    â¹142    5   
189870  NIVEA Soft Light Moisturizer for Face, Hand & ...    â¹142    5   
189871  NIVEA Soft Light Moisturizer for Face, Hand & ...    â¹142    5   
189872  NIVEA Soft Light Moisturizer for Face, Hand & ...    â¹142    4   
189873                                          

In [22]:
import nltk
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [23]:
from nltk.corpus import stopwords

# Assuming 'df' is your DataFrame and 'column_name' is the name of the column you want to process

# Get the list of stopwords
stop_words = set(stopwords.words('english'))

# Remove stopwords from the tokenized text in the column
new_data['Summary'] = new_data['Summary'].apply(lambda x: [word for word in x if word.lower() not in stop_words])

# Print the DataFrame to verify the changes
print(new_data)


                                              ProductName     Price Rate  \
0       Candes 12 L Room/Personal Air Cooler?ÿ?ÿ(White...  ??3,999    5   
1       Candes 12 L Room/Personal Air Cooler?ÿ?ÿ(White...  ??3,999    5   
2       Candes 12 L Room/Personal Air Cooler?ÿ?ÿ(White...  ??3,999    3   
3       Candes 12 L Room/Personal Air Cooler?ÿ?ÿ(White...  ??3,999    1   
4       Candes 12 L Room/Personal Air Cooler?ÿ?ÿ(White...  ??3,999    3   
...                                                   ...       ...  ...   
189869  NIVEA Soft Light Moisturizer for Face, Hand & ...    â¹142    5   
189870  NIVEA Soft Light Moisturizer for Face, Hand & ...    â¹142    5   
189871  NIVEA Soft Light Moisturizer for Face, Hand & ...    â¹142    5   
189872  NIVEA Soft Light Moisturizer for Face, Hand & ...    â¹142    4   
189873                                                         NaN  NaN   

                 Review                                            Summary  \
0        

In [24]:
new_data


,ProductName,Price,Rate,Review,Summary,Category
0,Candes 12 L Room/Personal Air Cooler?ÿ?ÿ(White...,"??3,999",5,Super!,"[Great, cooler.., excellent, air, flow, price,...",household
1,Candes 12 L Room/Personal Air Cooler?ÿ?ÿ(White...,"??3,999",5,Awesome,"[Best, budget, fit, cooler, ., Nice, cooling]",household
2,Candes 12 L Room/Personal Air Cooler?ÿ?ÿ(White...,"??3,999",3,Fair,"[quality, good, power, air, decent]",household
3,Candes 12 L Room/Personal Air Cooler?ÿ?ÿ(White...,"??3,999",1,Useless product,"[bad, product, 's, fan]",household
4,Candes 12 L Room/Personal Air Cooler?ÿ?ÿ(White...,"??3,999",3,Fair,"[Ok, ok, product]",household
...,...,...,...,...,...,...
189869,"NIVEA Soft Light Moisturizer for Face, Hand & ...",â¹142,5,Terrific,[Thanks],beauty & personal care
189870,"NIVEA Soft Light Moisturizer for Face, Hand & ...",â¹142,5,Terrific,[Good],beauty & personal care
189871,"NIVEA Soft Light Moisturizer for Face, Hand & ...",â¹142,5,Just wow!,[Super],beauty & personal care
189872,"NIVEA Soft Light Moisturizer for Face, Hand & ...",â¹142,4,Worth the money,[Good],beauty & personal care


In [25]:
import nltk
nltk.download('wordnet')




[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [26]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from textblob import TextBlob

# Assuming 'new_data' is your DataFrame containing 'Summary' column
# Step 1: Convert the 'Summary' column to strings if it's not already
new_data['Summary'] = new_data['Summary'].astype(str)

# Step 2: Apply sentiment analysis
new_data['Sentiment'] = new_data['Summary'].apply(lambda x: TextBlob(x).sentiment.polarity)

# Step 3: Assign sentiment labels based on polarity
new_data['Sentiment_Label'] = new_data['Sentiment'].apply(lambda x: 1 if x >= 0 else 0)

# Step 4: Initialize the TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=1000)

# Step 5: Fit and transform the summary texts to create a vector representation
tfidf_matrix = tfidf_vectorizer.fit_transform(new_data['Summary'])

# Step 6: Convert the TF-IDF matrix to a DataFrame (optional)
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_vectorizer.get_feature_names_out())

# Step 7: Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(tfidf_df, new_data['Sentiment_Label'], test_size=0.2, random_state=42)

# Step 8: Initialize the Naive Bayes classifier
nb_classifier = MultinomialNB()

# Step 9: Train the classifier on the training data
nb_classifier.fit(X_train, y_train)

# Step 10: Make predictions on the testing data
y_pred = nb_classifier.predict(X_test)

# Step 11: Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Accuracy: 0.9692955892034233


In [27]:
# user_input = input("Enter the text for sentiment analysis: ")

# # Vectorize the user input using the same TF-IDF vectorizer
# user_input_vectorized = tfidf_vectorizer.transform([user_input])

# # Predict the sentiment polarity of the user input
# predicted_sentiment = nb_classifier.predict(user_input_vectorized)

# # Output the predicted sentiment
# if predicted_sentiment[0] == 1:
#     print("The sentiment of the input text is Positive.")
    
# else:
#     print("The sentiment of the input text is Negative.")

In [28]:
new_data.to_csv('/kaggle/working/new_data.csv', index=False)

In [29]:
import pandas as pd
from textblob import TextBlob

# Load your existing data from an Excel file (replace 'existing_data.xlsx' with your file path)
existing_data = new_data

# Function to analyze sentiment
def analyze_sentiment(text):
    # Analyze sentiment using TextBlob
    sentiment_score = TextBlob(text).sentiment.polarity
    
    # Determine sentiment label
    if sentiment_score > 0:
        sentiment_label = "Positive"
    elif sentiment_score < 0:
        sentiment_label = "Negative"
    else:
        sentiment_label = "Neutral"
    
    return sentiment_label

# Apply sentiment analysis to each summary in the data
existing_data['Sentiment'] = existing_data['Summary'].apply(analyze_sentiment)

# Save the updated data to a new CSV file (replace '/kaggle/working/new_data_updated.csv' with your desired file path)
existing_data.to_csv('/kaggle/working/new_data_updated.csv', index=False)


In [30]:
existing_data

,ProductName,Price,Rate,Review,Summary,Category,Sentiment,Sentiment_Label
0,Candes 12 L Room/Personal Air Cooler?ÿ?ÿ(White...,"??3,999",5,Super!,"['Great', 'cooler..', 'excellent', 'air', 'flo...",household,Positive,1
1,Candes 12 L Room/Personal Air Cooler?ÿ?ÿ(White...,"??3,999",5,Awesome,"['Best', 'budget', 'fit', 'cooler', '.', 'Nice...",household,Positive,1
2,Candes 12 L Room/Personal Air Cooler?ÿ?ÿ(White...,"??3,999",3,Fair,"['quality', 'good', 'power', 'air', 'decent']",household,Positive,1
3,Candes 12 L Room/Personal Air Cooler?ÿ?ÿ(White...,"??3,999",1,Useless product,"['bad', 'product', ""'s"", 'fan']",household,Negative,0
4,Candes 12 L Room/Personal Air Cooler?ÿ?ÿ(White...,"??3,999",3,Fair,"['Ok', 'ok', 'product']",household,Positive,1
...,...,...,...,...,...,...,...,...
189869,"NIVEA Soft Light Moisturizer for Face, Hand & ...",â¹142,5,Terrific,['Thanks'],beauty & personal care,Positive,1
189870,"NIVEA Soft Light Moisturizer for Face, Hand & ...",â¹142,5,Terrific,['Good'],beauty & personal care,Positive,1
189871,"NIVEA Soft Light Moisturizer for Face, Hand & ...",â¹142,5,Just wow!,['Super'],beauty & personal care,Positive,1
189872,"NIVEA Soft Light Moisturizer for Face, Hand & ...",â¹142,4,Worth the money,['Good'],beauty & personal care,Positive,1


In [31]:
import pandas as pd

# Load your DataFrame (replace 'data.csv' with your file path)
data = existing_data

def calculate_average_sentiment(product_name):
    # Filter DataFrame based on the product name
    filtered_data = data[data['ProductName'] == product_name]
    
    # Calculate the average sentiment label
    average_sentiment = filtered_data['Sentiment_Label'].mean()
    
    return average_sentiment


In [32]:
import pandas as pd

# Load your DataFrame (replace 'data.csv' with your file path)
data = existing_data
def calculate_average_sentiment(product_name):
    # Filter DataFrame based on the product name
    filtered_data = data[data['ProductName'] == product_name]
    
    # Calculate the average sentiment label
    average_sentiment = filtered_data['Sentiment_Label'].mean()
    
    return average_sentiment

# Create a list to store the data
rows = []

# Get unique product names
unique_product_names = data['ProductName'].unique()

# Iterate over each product name
for product_name in unique_product_names:
    # Filter DataFrame based on the product name
    filtered_data = data[data['ProductName'] == product_name]
    
    # Calculate the average sentiment label
    average_sentiment = calculate_average_sentiment(product_name)
    
    # Take the first row of filtered data to extract product category
    category = filtered_data['Category'].iloc[0]
    
    # Take the first review and summary of the filtered data
    review = filtered_data['Review'].iloc[0]
    summary = filtered_data['Summary'].iloc[0]
    
    # Append the data to the list
    rows.append({'ProductName': product_name,
                 'Category': category,
                 'Review': review,
                 'Summary': summary,
                 'Average_Sentiment_Label': average_sentiment})

# Create a DataFrame from the list of dictionaries
new_data = pd.DataFrame(rows)

# Save the new DataFrame to a CSV file
new_data.to_csv('/kaggle/working/new_data_final.csv', index=False)


In [33]:
import pandas as pd

# Load your DataFrame (replace 'data.csv' with your file path)
new_data = pd.read_csv('/kaggle/working/new_data_final.csv')

# Extract the unique categories
unique_categories = data['Category'].unique()

# Print the list of unique categories
print("List of unique categories:")
for category in unique_categories:
    print(category)


List of unique categories:
household
electronics
baby
automotive
arts & crafts
clothing
beauty & personal care
books
sports & outdoors
tools & home improvement
health
toys & games
Uncategorized


In [34]:
# List of categories
categories = [
    'household',
    'electronics',
    'baby',
    'automotive',
    'arts & crafts',
    'clothing',
    'beauty & personal care',
    'books',
    'sports & outdoors',
    'tools & home improvement',
    'health',
    'toys & games'
]

# Display the list of categories to the user
print("Select a category:")
for index, category in enumerate(categories, start=1):
    print(f"{index}. {category}")

# Prompt the user to select a category
selected_index = int(input("Enter the number corresponding to your selected category: ")) - 1

# Validate the user input
if selected_index < 0 or selected_index >= len(categories):
    print("Invalid selection. Please enter a number within the provided range.")
else:
    selected_category = categories[selected_index]
    print(f"You selected: {selected_category}")

    # Prompt the user to input their desired item
    desired_item = input("Enter what you are looking for: ")
    print(f"You are looking for: {desired_item}")


Select a category:
1. household
2. electronics
3. baby
4. automotive
5. arts & crafts
6. clothing
7. beauty & personal care
8. books
9. sports & outdoors
10. tools & home improvement
11. health
12. toys & games


Enter the number corresponding to your selected category:  12


You selected: toys & games


Enter what you are looking for:  helicopter


You are looking for: helicopter


In [35]:
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Embedding, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load the data
data = pd.read_csv('/kaggle/working/new_data_final.csv')

# Drop rows with missing 'Review' values
data = data.dropna(subset=['Review'])

# Tokenization
tokenizer = Tokenizer(num_words=1000)
tokenizer.fit_on_texts(data['Review'])
X = tokenizer.texts_to_sequences(data['Review'])
X = pad_sequences(X, maxlen=100)  # Assuming max sequence length of 100

# Prepare labels
y = data['Average_Sentiment_Label']


# LSTM Model
model = Sequential()
model.add(Embedding(input_dim=1000, output_dim=100))  # Embedding layer
model.add(LSTM(units=64, dropout=0.2, recurrent_dropout=0.2))  # LSTM layer
model.add(Dense(1, activation='sigmoid'))  # Output layer

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X, y, batch_size=64, epochs=5)

# Save the model
model.save('/kaggle/working/sentiment_analysis_model.h5')
print("Model saved successfully.")


Epoch 1/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 8s 141ms/step - accuracy: 0.0716 - loss: 0.5488
Epoch 2/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 137ms/step - accuracy: 0.0794 - loss: 0.3542
Epoch 3/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 140ms/step - accuracy: 0.0797 - loss: 0.3520
Epoch 4/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 146ms/step - accuracy: 0.0956 - loss: 0.3419
Epoch 5/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 145ms/step - accuracy: 0.1000 - loss: 0.3321
Model saved successfully.


In [36]:
# from tensorflow.keras.models import load_model

# # Load the trained model
# model = load_model('/kaggle/working/sentiment_analysis_model.h5')

# # List of categories to choose from
# categories = [
#     'household',
#     'electronics',
#     'baby',
#     'automotive',
#     'arts & crafts',
#     'clothing',
#     'beauty & personal care',
#     'books',
#     'sports & outdoors',
#     'tools & home improvement',
#     'health',
#     'toys & games'
# ]

# # Print available categories for user to choose from
# print("Available categories to choose from:")
# for index, category in enumerate(categories, start=1):
#     print(f"{index}. {category}")

# # Get user input for category
# category_index = int(input("Enter the number corresponding to the category you are interested in: "))
# selected_category = categories[category_index - 1]

# # Get user input for product name
# product = input("Enter the product name you are looking for: ")

# # Filter data based on user input category and product name
# filtered_data = data[(data['Category'] == selected_category) & (data['ProductName'].str.contains(product, case=False))]

# # Tokenize user input
# sequences = tokenizer.texts_to_sequences(filtered_data['Review'])
# padded_sequences = pad_sequences(sequences, maxlen=100)

# # Predict sentiment labels
# predicted_labels = model.predict(padded_sequences)

# # Add predicted labels to filtered data
# filtered_data['Predicted_Sentiment_Label'] = predicted_labels

# # Sort products based on average sentiment labels in descending order
# sorted_products = filtered_data.sort_values(by='Average_Sentiment_Label', ascending=False)

# # Print suggested products
# print("\nSuggested products:")
# print(sorted_products[['ProductName', 'Summary', 'Average_Sentiment_Label']])


In [37]:
# from tensorflow.keras.models import load_model

# # Load the trained model
# model = load_model('/kaggle/working/sentiment_analysis_model.h5')

# # List of categories to choose from
# categories = [
#     'household',
#     'electronics',
#     'baby',
#     'automotive',
#     'arts & crafts',
#     'clothing',
#     'beauty & personal care',
#     'books',
#     'sports & outdoors',
#     'tools & home improvement',
#     'health',
#     'toys & games'
# ]

# # Print available categories for user to choose from
# print("Available categories to choose from:")
# for index, category in enumerate(categories, start=1):
#     print(f"{index}. {category}")

# # Get user input for category
# category_index = int(input("Enter the number corresponding to the category you are interested in: "))
# selected_category = categories[category_index - 1]

# # Get user input for product name
# product = input("Enter the product name you are looking for: ")

# # Filter data based on user input category and product name
# filtered_data = data[(data['Category'] == selected_category) & (data['ProductName'].str.contains(product, case=False))]

# # Tokenize user input
# sequences = tokenizer.texts_to_sequences(filtered_data['Review'])
# padded_sequences = pad_sequences(sequences, maxlen=100)

# # Predict sentiment labels
# predicted_labels = model.predict(padded_sequences)

# # Add predicted labels to filtered data
# filtered_data['Predicted_Sentiment_Label'] = predicted_labels

# # Sort products based on average sentiment labels in descending order
# sorted_products = filtered_data.sort_values(by='Average_Sentiment_Label', ascending=False)

# # Iterate over the sorted products DataFrame and print relevant recommendations
# for index, product_row in sorted_products.iterrows():
#     product_name = product_row['ProductName']
#     summary = product_row['Summary']
#     avg_sentiment = product_row['Average_Sentiment_Label'] * 100  # Multiply by 100 to get percentage
#     print(f"Product Name: {product_name}")
# #     print(f"Summary: {summary}")
#     print(f"User who liked the product: {avg_sentiment:.2f}%\n")  # Format sentiment as percentage with 2 decimal places


In [38]:
from tensorflow.keras.models import load_model

# Load the trained model
model = load_model('/kaggle/working/sentiment_analysis_model.h5')

# List of categories to choose from
categories = [
    'household',
    'electronics',
    'baby',
    'automotive',
    'arts & crafts',
    'clothing',
    'beauty & personal care',
    'books',
    'sports & outdoors',
    'tools & home improvement',
    'health',
    'toys & games'
]

# Print available categories for user to choose from
print("Available categories to choose from:")
for index, category in enumerate(categories, start=1):
    print(f"{index}. {category}")

# Get user input for category
category_index = int(input("Enter the number corresponding to the category you are interested in: "))
selected_category = categories[category_index - 1]

# Get user input for product name
product = input("Enter the product name you are looking for: ")

# Filter data based on user input category and product name
filtered_data = data[(data['Category'] == selected_category) & (data['ProductName'].str.contains(product, case=False))]

# Tokenize user input
sequences = tokenizer.texts_to_sequences(filtered_data['Review'])
padded_sequences = pad_sequences(sequences, maxlen=100)

# Predict sentiment labels
predicted_labels = model.predict(padded_sequences)

# Add predicted labels to filtered data
filtered_data['Predicted_Sentiment_Label'] = predicted_labels

# Sort products based on average sentiment labels in descending order
sorted_products = filtered_data.sort_values(by='Average_Sentiment_Label', ascending=False)

# Print all relevant products
if len(sorted_products) == 0:
    print("No relevant products found.")
else:
    print("\nRelevant products (sorted by average sentiment label in descending order):")
    for index, product_row in sorted_products.iterrows():
        product_name = product_row['ProductName']
        summary = product_row['Summary']
        avg_sentiment = product_row['Average_Sentiment_Label'] * 100  # Multiply by 100 to get percentage
        print(f"Product Name: {product_name}")
#         print(f"Summary: {summary}")
        print(f"User who liked the product: {avg_sentiment:.2f}%\n")  # Format sentiment as percentage with 2 decimal places


Available categories to choose from:
1. household
2. electronics
3. baby
4. automotive
5. arts & crafts
6. clothing
7. beauty & personal care
8. books
9. sports & outdoors
10. tools & home improvement
11. health
12. toys & games


Enter the number corresponding to the category you are interested in:  12
Enter the product name you are looking for:  Helicopter


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 228ms/step

Relevant products (sorted by average sentiment label in descending order):
Product Name: Mayne Flying Remote Control RC Induction Type 2-in-1 Indoor Outdoor Helicopter?ÿ?ÿ(Multicolor)
User who liked the product: 72.27%



/tmp/ipykernel_34/3186809009.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['Predicted_Sentiment_Label'] = predicted_labels


In [39]:
# import pandas as pd
# from textblob import TextBlob

# # Load your existing data from an Excel file (replace 'existing_data.xlsx' with your file path)
# existing_data = new_data

# # Function to analyze sentiment
# def analyze_sentiment(text):
#     # Analyze sentiment using TextBlob
#     sentiment_score = TextBlob(text).sentiment.polarity
    
#     # Determine sentiment label
#     if sentiment_score > 0:
#         sentiment_label = "Positive"
#     elif sentiment_score < 0:
#         sentiment_label = "Negative"
#     else:
#         sentiment_label = "Neutral"
    
#     return sentiment_label

# # Apply sentiment analysis to each summary in the data
# existing_data['Sentiment'] = existing_data['Review'].apply(analyze_sentiment)

# # Save the updated data to a new CSV file (replace '/kaggle/working/new_data_updated.csv' with your desired file path)
# existing_data.to_csv('/kaggle/working/new_data_updated.csv', index=False)


In [40]:
from textblob import TextBlob

# Function to analyze sentiment and print result
def analyze_sentiment(text):
    # Analyze sentiment using TextBlob
    sentiment_score = TextBlob(text).sentiment.polarity
    
    # Determine sentiment label
    if sentiment_score > 0:
        sentiment_label = "Positive"
    elif sentiment_score < 0:
        sentiment_label = "Negative"
    else:
        sentiment_label = "Neutral"
    
    # Print result
    print(f"Sentiment: {sentiment_label}")

# Get input from user
user_input = input("Enter text: ")

# Analyze sentiment and print result
analyze_sentiment(user_input)


Enter text:  BAD


Sentiment: Negative
